<a href="https://colab.research.google.com/github/TobiPrae/customer_segmentation/blob/main/CustomerSegmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import packages

In [48]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)
import matplotlib.pyplot as plt
import seaborn as sns

# magic word for producing visualizations in notebook
%matplotlib inline
print("Libraries successfully imported")

Libraries successfully imported


In [49]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

print("Everything successfully set up")

Everything successfully set up


# Load data

In [45]:
# Get list of every available file in google drive
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: meta_df.csv, id: 1Phry8vukY_1Dp-weiHY5288jhwXEb1xG
title: information_levels_attributes.xlsx, id: 1icPBxHTV0zysKvQSJhSDPDhyH8Qd3uqW
title: attributes_values.xlsx, id: 1wFNhCQm7Cv9CGJ7Pm_RbtrJdnLb4zyuC
title: azdias.csv, id: 135UcsbUyLcwS16QHa7lFcXHKhuy7HwWF
title: mailout_train.csv, id: 1YclR47YPa6Q5S98bvMNflPcb8h0LQC_I
title: customers.csv, id: 1eDq76GDZLXeonowxPrbwF-EHZwvaENLK
title: mailout_test.csv, id: 1mpHKYwfFVuQ1kBQroxL3HYnMCcMxfH9_
title: ds_nanodegree, id: 1edNrXjcSjQd4ZdpZl304TxtErX44EhZi
title: Commanders, id: 1ztnLmAapmoTfb_vXewqlyxnnq8kd6Jll1KRzOjU3jRU
title: Unbenanntes Dokument, id: 1W81POmANip3vfQ5PAb4kApxLuxZrkbKoWgScqdl-5Fs
title: deck (4).pdf, id: 1aTWzCZtbeXAlio2tuwwrg2RNNks5a_jy
title: Präsentation (2), id: 1W5CdUsrSmqqIErZHXEV3pRf88nGuRs0KRavmCD_y2_g
title: Präsentation (2).pptx, id: 1UM4GE9QYBG1PLX_917j4QddUB-RYmQMt
title: IS_Healthcare.xlsx, id: 1FHw6yKlGlNdTEUUQ5j9qWMDAB0dTvtmP
title: Pokes, id: 1XZCoDrPqFGK8MLz2E5qbGKxa2BWgJn-6fqElgrDpmyM
title: Proj

In [4]:
# download demographic data
download_data = drive.CreateFile({'id': '135UcsbUyLcwS16QHa7lFcXHKhuy7HwWF'})
download_data.GetContentFile('azdias.csv')  
df_azdias = pd.read_csv('azdias.csv', engine="python", sep=';')

print(df_azdias.shape)

(891221, 366)


In [5]:
# download customer data
download_data = drive.CreateFile({'id': '1eDq76GDZLXeonowxPrbwF-EHZwvaENLK'})
download_data.GetContentFile('customers.csv')  
df_customers = pd.read_csv('customers.csv', engine="python", sep=';')

print(df_customers.shape)

(191652, 369)


In [6]:
# download attribute values and clean them
download_data = drive.CreateFile({'id': '1wFNhCQm7Cv9CGJ7Pm_RbtrJdnLb4zyuC'})
download_data.GetContentFile('attributes_values.xlsx')  
df_attributes_values = pd.read_excel('attributes_values.xlsx', names=["None", "attribute", "description", "value", "meaning"])
df_attributes_values = df_attributes_values.drop(columns=['None'])[1:]
print(df_attributes_values.shape)

(2258, 4)


In [7]:
# download attribute information levels and clean them
download_data = drive.CreateFile({'id': '1icPBxHTV0zysKvQSJhSDPDhyH8Qd3uqW'})
download_data.GetContentFile('information_levels_attributes.xlsx')  
df_information_levels = pd.read_excel('information_levels_attributes.xlsx', names=["None", "Information_Level", "attribute", "description", "additional_notes"])
df_information_levels = df_information_levels.drop(columns=["None"])[1:]
print(df_information_levels.shape)

(313, 4)


# Functions

In [8]:
def get_description(col):
  '''
  Gets description for given column.

  Args:
  - col: column name

  Returns:
  - description or "No description available."
  '''
  if(df_attributes_values.loc[df_attributes_values["attribute"] == col].shape[0] == 0):
    return "No description available."
  else:
    return df_attributes_values.loc[df_attributes_values["attribute"] == col]["description"].values[0]

In [9]:
def engineer_CAMEO_INTL_2015(x):
  '''
  Transform value

  Args:
  - x: Cell value

  Returns:
  - Transformed x

  '''
  if((x == 'X') | (x == 'XX')):
    return np.nan
  if(str(x)[0] == '1'):
    return '1'
  if(str(x)[0] == '2'):
    return '2'
  if(str(x)[0] == '3'):
    return '3'
  if(str(x)[0] == '4'):
    return '4'
  if(str(x)[0] == '5'):
    return '5'
  else:
    return np.nan

In [10]:
def engineer_OST_WEST_KZ(x):
  if(x == 'W'):
    return 1
  elif(x == 'O'):
    return 0
  else:
    return np.nan

# Part 1: Customer Segmentation Report

### Quick overview

In [11]:
df_azdias.head(5)

,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,ANZ_HH_TITEL,ANZ_KINDER,ANZ_PERSONEN,ANZ_STATISTISCHE_HAUSHALTE,ANZ_TITEL,ARBEIT,BALLRAUM,CAMEO_DEU_2015,CAMEO_DEUG_2015,CAMEO_INTL_2015,CJT_GESAMTTYP,CJT_KATALOGNUTZER,CJT_TYP_1,CJT_TYP_2,CJT_TYP_3,CJT_TYP_4,CJT_TYP_5,CJT_TYP_6,D19_BANKEN_ANZ_12,D19_BANKEN_ANZ_24,D19_BANKEN_DATUM,D19_BANKEN_DIREKT,D19_BANKEN_GROSS,D19_BANKEN_LOKAL,D19_BANKEN_OFFLINE_DATUM,D19_BANKEN_ONLINE_DATUM,D19_BANKEN_ONLINE_QUOTE_12,D19_BANKEN_REST,D19_BEKLEIDUNG_GEH,D19_BEKLEIDUNG_REST,D19_BILDUNG,D19_BIO_OEKO,D19_BUCH_CD,D19_DIGIT_SERV,D19_DROGERIEARTIKEL,D19_ENERGIE,D19_FREIZEIT,D19_GARTEN,D19_GESAMT_ANZ_12,D19_GESAMT_ANZ_24,D19_GESAMT_DATUM,D19_GESAMT_OFFLINE_DATUM,D19_GESAMT_ONLINE_DATUM,D19_GESAMT_ONLINE_QUOTE_12,D19_HANDWERK,D19_HAUS_DEKO,D19_KINDERARTIKEL,D19_KONSUMTYP,D19_KONSUMTYP_MAX,D19_KOSMETIK,D19_LEBENSMITTEL,D19_LETZTER_KAUF_BRANCHE,D19_LOTTO,D19_NAHRUNGSERGAENZUNG,D19_RATGEBER,D19_REISEN,D19_SAMMELARTIKEL,D19_SCHUHE,D19_SONSTIGE,D19_SOZIALES,D19_TECHNIK,D19_TELKO_ANZ_12,D19_TELKO_ANZ_24,D19_TELKO_DATUM,D19_TELKO_MOBILE,D19_TELKO_OFFLINE_DATUM,D19_TELKO_ONLINE_DATUM,D19_TELKO_ONLINE_QUOTE_12,D19_TELKO_REST,D19_TIERARTIKEL,D19_VERSAND_ANZ_12,D19_VERSAND_ANZ_24,D19_VERSAND_DATUM,D19_VERSAND_OFFLINE_DATUM,D19_VERSAND_ONLINE_DATUM,D19_VERSAND_ONLINE_QUOTE_12,D19_VERSAND_REST,D19_VERSI_ANZ_12,D19_VERSI_ANZ_24,D19_VERSI_DATUM,D19_VERSI_OFFLINE_DATUM,D19_VERSI_ONLINE_DATUM,D19_VERSI_ONLINE_QUOTE_12,D19_VERSICHERUNGEN,D19_VOLLSORTIMENT,D19_WEIN_FEINKOST,DSL_FLAG,EINGEFUEGT_AM,EINGEZOGENAM_HH_JAHR,EWDICHTE,EXTSEL992,FINANZ_ANLEGER,FINANZ_HAUSBAUER,FINANZ_MINIMALIST,FINANZ_SPARER,FINANZ_UNAUFFAELLIGER,FINANZ_VORSORGER,FINANZTYP,FIRMENDICHTE,GEBAEUDETYP,GEBAEUDETYP_RASTER,GEBURTSJAHR,GEMEINDETYP,GFK_URLAUBERTYP,GREEN_AVANTGARDE,HEALTH_TYP,HH_DELTA_FLAG,HH_EINKOMMEN_SCORE,INNENSTADT,KBA05_ALTER1,KBA05_ALTER2,KBA05_ALTER3,KBA05_ALTER4,KBA05_ANHANG,KBA05_ANTG1,KBA05_ANTG2,KBA05_ANTG3,KBA05_ANTG4,KBA05_AUTOQUOT,KBA05_BAUMAX,KBA05_CCM1,KBA05_CCM2,KBA05_CCM3,KBA05_CCM4,KBA05_DIESEL,KBA05_FRAU,KBA05_GBZ,KBA05_HERST1,KBA05_HERST2,KBA05_HERST3,KBA05_HERST4,KBA05_HERST5,KBA05_HERSTTEMP,KBA05_KRSAQUOT,KBA05_KRSHERST1,KBA05_KRSHERST2,KBA05_KRSHERST3,KBA05_KRSKLEIN,KBA05_KRSOBER,KBA05_KRSVAN,KBA05_KRSZUL,KBA05_KW1,KBA05_KW2,KBA05_KW3,KBA05_MAXAH,KBA05_MAXBJ,KBA05_MAXHERST,KBA05_MAXSEG,KBA05_MAXVORB,KBA05_MOD1,KBA05_MOD2,KBA05_MOD3,KBA05_MOD4,KBA05_MOD8,KBA05_MODTEMP,KBA05_MOTOR,KBA05_MOTRAD,KBA05_SEG1,KBA05_SEG10,KBA05_SEG2,KBA05_SEG3,KBA05_SEG4,KBA05_SEG5,KBA05_SEG6,KBA05_SEG7,KBA05_SEG8,KBA05_SEG9,KBA05_VORB0,KBA05_VORB1,KBA05_VORB2,KBA05_ZUL1,KBA05_ZUL2,KBA05_ZUL3,KBA05_ZUL4,KBA13_ALTERHALTER_30,KBA13_ALTERHALTER_45,KBA13_ALTERHALTER_60,KBA13_ALTERHALTER_61,KBA13_ANTG1,KBA13_ANTG2,KBA13_ANTG3,KBA13_ANTG4,KBA13_ANZAHL_PKW,KBA13_AUDI,KBA13_AUTOQUOTE,KBA13_BAUMAX,KBA13_BJ_1999,KBA13_BJ_2000,KBA13_BJ_2004,KBA13_BJ_2006,KBA13_BJ_2008,KBA13_BJ_2009,KBA13_BMW,KBA13_CCM_0_1400,KBA13_CCM_1000,KBA13_CCM_1200,KBA13_CCM_1400,KBA13_CCM_1401_2500,KBA13_CCM_1500,KBA13_CCM_1600,KBA13_CCM_1800,KBA13_CCM_2000,KBA13_CCM_2500,KBA13_CCM_2501,KBA13_CCM_3000,KBA13_CCM_3001,KBA13_FAB_ASIEN,KBA13_FAB_SONSTIGE,KBA13_FIAT,KBA13_FORD,KBA13_GBZ,KBA13_HALTER_20,KBA13_HALTER_25,KBA13_HALTER_30,KBA13_HALTER_35,KBA13_HALTER_40,KBA13_HALTER_45,KBA13_HALTER_50,KBA13_HALTER_55,KBA13_HALTER_60,KBA13_HALTER_65,KBA13_HALTER_66,KBA13_HERST_ASIEN,KBA13_HERST_AUDI_VW,KBA13_HERST_BMW_BENZ,KBA13_HERST_EUROPA,KBA13_HERST_FORD_OPEL,KBA13_HERST_SONST,KBA13_HHZ,KBA13_KMH_0_140,KBA13_KMH_110,KBA13_KMH_140,KBA13_KMH_140_210,KBA13_KMH_180,KBA13_KMH_210,KBA13_KMH_211,KBA13_KMH_250,KBA13_KMH_251,KBA13_KRSAQUOT,KBA13_KRSHERST_AUDI_VW,KBA13_KRSHERST_BMW_BENZ,KBA13_KRSHERST_FORD_OPEL,KBA13_KRSSEG_KLEIN,KBA13_KRSSEG_OBER,KBA13_KRSSEG_VAN,KBA13_KRSZUL_NEU,KBA13_KW_0_60,KBA13_KW_110,KBA13_KW_120,KBA13_KW_121,KBA13_KW_30,KBA13_KW_40,KBA13_KW_50,KBA13_KW_60,KBA13_KW_61_120,KBA13_KW_70,KBA13_KW_80,

In [12]:
df_customers.head(5)

,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,ANZ_HH_TITEL,ANZ_KINDER,ANZ_PERSONEN,ANZ_STATISTISCHE_HAUSHALTE,ANZ_TITEL,ARBEIT,BALLRAUM,CAMEO_DEU_2015,CAMEO_DEUG_2015,CAMEO_INTL_2015,CJT_GESAMTTYP,CJT_KATALOGNUTZER,CJT_TYP_1,CJT_TYP_2,CJT_TYP_3,CJT_TYP_4,CJT_TYP_5,CJT_TYP_6,D19_BANKEN_ANZ_12,D19_BANKEN_ANZ_24,D19_BANKEN_DATUM,D19_BANKEN_DIREKT,D19_BANKEN_GROSS,D19_BANKEN_LOKAL,D19_BANKEN_OFFLINE_DATUM,D19_BANKEN_ONLINE_DATUM,D19_BANKEN_ONLINE_QUOTE_12,D19_BANKEN_REST,D19_BEKLEIDUNG_GEH,D19_BEKLEIDUNG_REST,D19_BILDUNG,D19_BIO_OEKO,D19_BUCH_CD,D19_DIGIT_SERV,D19_DROGERIEARTIKEL,D19_ENERGIE,D19_FREIZEIT,D19_GARTEN,D19_GESAMT_ANZ_12,D19_GESAMT_ANZ_24,D19_GESAMT_DATUM,D19_GESAMT_OFFLINE_DATUM,D19_GESAMT_ONLINE_DATUM,D19_GESAMT_ONLINE_QUOTE_12,D19_HANDWERK,D19_HAUS_DEKO,D19_KINDERARTIKEL,D19_KONSUMTYP,D19_KONSUMTYP_MAX,D19_KOSMETIK,D19_LEBENSMITTEL,D19_LETZTER_KAUF_BRANCHE,D19_LOTTO,D19_NAHRUNGSERGAENZUNG,D19_RATGEBER,D19_REISEN,D19_SAMMELARTIKEL,D19_SCHUHE,D19_SONSTIGE,D19_SOZIALES,D19_TECHNIK,D19_TELKO_ANZ_12,D19_TELKO_ANZ_24,D19_TELKO_DATUM,D19_TELKO_MOBILE,D19_TELKO_OFFLINE_DATUM,D19_TELKO_ONLINE_DATUM,D19_TELKO_ONLINE_QUOTE_12,D19_TELKO_REST,D19_TIERARTIKEL,D19_VERSAND_ANZ_12,D19_VERSAND_ANZ_24,D19_VERSAND_DATUM,D19_VERSAND_OFFLINE_DATUM,D19_VERSAND_ONLINE_DATUM,D19_VERSAND_ONLINE_QUOTE_12,D19_VERSAND_REST,D19_VERSI_ANZ_12,D19_VERSI_ANZ_24,D19_VERSI_DATUM,D19_VERSI_OFFLINE_DATUM,D19_VERSI_ONLINE_DATUM,D19_VERSI_ONLINE_QUOTE_12,D19_VERSICHERUNGEN,D19_VOLLSORTIMENT,D19_WEIN_FEINKOST,DSL_FLAG,EINGEFUEGT_AM,EINGEZOGENAM_HH_JAHR,EWDICHTE,EXTSEL992,FINANZ_ANLEGER,FINANZ_HAUSBAUER,FINANZ_MINIMALIST,FINANZ_SPARER,FINANZ_UNAUFFAELLIGER,FINANZ_VORSORGER,FINANZTYP,FIRMENDICHTE,GEBAEUDETYP,GEBAEUDETYP_RASTER,GEBURTSJAHR,GEMEINDETYP,GFK_URLAUBERTYP,GREEN_AVANTGARDE,HEALTH_TYP,HH_DELTA_FLAG,HH_EINKOMMEN_SCORE,INNENSTADT,KBA05_ALTER1,KBA05_ALTER2,KBA05_ALTER3,KBA05_ALTER4,KBA05_ANHANG,KBA05_ANTG1,KBA05_ANTG2,KBA05_ANTG3,KBA05_ANTG4,KBA05_AUTOQUOT,KBA05_BAUMAX,KBA05_CCM1,KBA05_CCM2,KBA05_CCM3,KBA05_CCM4,KBA05_DIESEL,KBA05_FRAU,KBA05_GBZ,KBA05_HERST1,KBA05_HERST2,KBA05_HERST3,KBA05_HERST4,KBA05_HERST5,KBA05_HERSTTEMP,KBA05_KRSAQUOT,KBA05_KRSHERST1,KBA05_KRSHERST2,KBA05_KRSHERST3,KBA05_KRSKLEIN,KBA05_KRSOBER,KBA05_KRSVAN,KBA05_KRSZUL,KBA05_KW1,KBA05_KW2,KBA05_KW3,KBA05_MAXAH,KBA05_MAXBJ,KBA05_MAXHERST,KBA05_MAXSEG,KBA05_MAXVORB,KBA05_MOD1,KBA05_MOD2,KBA05_MOD3,KBA05_MOD4,KBA05_MOD8,KBA05_MODTEMP,KBA05_MOTOR,KBA05_MOTRAD,KBA05_SEG1,KBA05_SEG10,KBA05_SEG2,KBA05_SEG3,KBA05_SEG4,KBA05_SEG5,KBA05_SEG6,KBA05_SEG7,KBA05_SEG8,KBA05_SEG9,KBA05_VORB0,KBA05_VORB1,KBA05_VORB2,KBA05_ZUL1,KBA05_ZUL2,KBA05_ZUL3,KBA05_ZUL4,KBA13_ALTERHALTER_30,KBA13_ALTERHALTER_45,KBA13_ALTERHALTER_60,KBA13_ALTERHALTER_61,KBA13_ANTG1,KBA13_ANTG2,KBA13_ANTG3,KBA13_ANTG4,KBA13_ANZAHL_PKW,KBA13_AUDI,KBA13_AUTOQUOTE,KBA13_BAUMAX,KBA13_BJ_1999,KBA13_BJ_2000,KBA13_BJ_2004,KBA13_BJ_2006,KBA13_BJ_2008,KBA13_BJ_2009,KBA13_BMW,KBA13_CCM_0_1400,KBA13_CCM_1000,KBA13_CCM_1200,KBA13_CCM_1400,KBA13_CCM_1401_2500,KBA13_CCM_1500,KBA13_CCM_1600,KBA13_CCM_1800,KBA13_CCM_2000,KBA13_CCM_2500,KBA13_CCM_2501,KBA13_CCM_3000,KBA13_CCM_3001,KBA13_FAB_ASIEN,KBA13_FAB_SONSTIGE,KBA13_FIAT,KBA13_FORD,KBA13_GBZ,KBA13_HALTER_20,KBA13_HALTER_25,KBA13_HALTER_30,KBA13_HALTER_35,KBA13_HALTER_40,KBA13_HALTER_45,KBA13_HALTER_50,KBA13_HALTER_55,KBA13_HALTER_60,KBA13_HALTER_65,KBA13_HALTER_66,KBA13_HERST_ASIEN,KBA13_HERST_AUDI_VW,KBA13_HERST_BMW_BENZ,KBA13_HERST_EUROPA,KBA13_HERST_FORD_OPEL,KBA13_HERST_SONST,KBA13_HHZ,KBA13_KMH_0_140,KBA13_KMH_110,KBA13_KMH_140,KBA13_KMH_140_210,KBA13_KMH_180,KBA13_KMH_210,KBA13_KMH_211,KBA13_KMH_250,KBA13_KMH_251,KBA13_KRSAQUOT,KBA13_KRSHERST_AUDI_VW,KBA13_KRSHERST_BMW_BENZ,KBA13_KRSHERST_FORD_OPEL,KBA13_KRSSEG_KLEIN,KBA13_KRSSEG_OBER,KBA13_KRSSEG_VAN,KBA13_KRSZUL_NEU,KBA13_KW_0_60,KBA13_KW_110,KBA13_KW_120,KBA13_KW_121,KBA13_KW_30,KBA13_KW_40,KBA13_KW_50,KBA13_KW_60,KBA13_KW_61_120,KBA13_KW_70,KBA13_KW_80,

In [13]:
# Drop ID
df_azdias = df_azdias.drop(columns="LNR")
df_customers = df_customers.drop(columns="LNR")

In [14]:
# Check df_information_levels
df_information_levels.head(5)

,Information_Level,attribute,description,additional_notes
1,NaN,AGER_TYP,best-ager typology,in cooperation with Kantar TNS; the informatio...
2,Person,ALTERSKATEGORIE_GROB,age through prename analysis,modelled on millions of first name-age-referen...
3,NaN,ANREDE_KZ,gender,NaN
4,NaN,CJT_GESAMTTYP,Customer-Journey-Typology relating to the pref...,"relating to the preferred information, marketi..."
5,NaN,FINANZ_MINIMALIST,financial typology: low financial interest,Gfk-Typology based on a representative househo...


In [15]:
# Check df_attributes_values
df_attributes_values.head(5)

,attribute,description,value,meaning
1,AGER_TYP,best-ager typology,-1,unknown
2,NaN,NaN,0,no classification possible
3,NaN,NaN,1,passive elderly
4,NaN,NaN,2,cultural elderly
5,NaN,NaN,3,experience-driven elderly


In [16]:
# Forward filling the attribute values df for better filtering/slicing
df_attributes_values["attribute"] = df_attributes_values["attribute"].fillna(method="ffill")
df_attributes_values["description"] = df_attributes_values["description"].fillna(method="ffill")
df_attributes_values.head(5)

,attribute,description,value,meaning
1,AGER_TYP,best-ager typology,-1,unknown
2,AGER_TYP,best-ager typology,0,no classification possible
3,AGER_TYP,best-ager typology,1,passive elderly
4,AGER_TYP,best-ager typology,2,cultural elderly
5,AGER_TYP,best-ager typology,3,experience-driven elderly


In [17]:
# Create a dataframe with meta information
meta_df = pd.DataFrame(df_azdias.columns.tolist())
meta_df = meta_df.rename(columns={0: "column_name"})
meta_df["na_percentage_azdias"] = meta_df["column_name"].apply(lambda x: round(df_azdias[x].isna().sum()/df_azdias.shape[0], 4))
meta_df["na_percentage_customers"] = meta_df["column_name"].apply(lambda x: round(df_customers[x].isna().sum()/df_customers.shape[0], 4))
meta_df["dtypes_azdias"] =  meta_df["column_name"].apply(lambda x: set(df_azdias[x].map(type).tolist()))
meta_df["dtypes_customers"] =  meta_df["column_name"].apply(lambda x: set(df_customers[x].map(type).tolist()))
meta_df["n_dtypes_azdias"] = meta_df["column_name"].apply(lambda x: len(set(df_azdias[x].map(type).tolist())))
meta_df["n_dtypes_customers"] = meta_df["column_name"].apply(lambda x: len(set(df_customers[x].map(type).tolist())))
meta_df["unique_values_azdias"] = meta_df["column_name"].apply(lambda x: len(df_azdias[x].value_counts()))
meta_df["unique_values_customers"] = meta_df["column_name"].apply(lambda x: len(df_customers[x].value_counts()))
meta_df["description"] =  meta_df["column_name"].apply(get_description)
meta_df["action"] = "Keep"
meta_df["reason"] = "None"
meta_df

,column_name,na_percentage_azdias,na_percentage_customers,dtypes_azdias,dtypes_customers,n_dtypes_azdias,n_dtypes_customers,unique_values_azdias,unique_values_customers,description,action,reason
0,AGER_TYP,0.0000,0.0000,{<class 'int'>},{<class 'int'>},1,1,5,5,best-ager typology,Keep,None
1,AKT_DAT_KL,0.0825,0.2431,{<class 'float'>},{<class 'float'>},1,1,9,9,No description available.,Keep,None
2,ALTER_HH,0.0825,0.2431,{<class 'float'>},{<class 'float'>},1,1,22,21,main age within the household,Keep,None
3,ALTER_KIND1,0.9090,0.9386,{<class 'float'>},{<class 'float'>},1,1,17,17,No description available.,Keep,None
4,ALTER_KIND2,0.9669,0.9734,{<class 'float'>},{<class 'float'>},1,1,17,17,No description available.,Keep,None
...,...,...,...,...,...,...,...,...,...,...,...,...
360,WOHNDAUER_2008,0.0825,0.2431,{<class 'float'>},{<class 'float'>},1,1,9,9,length of residence,Keep,None
361,WOHNLAGE,0.1045,0.2605,{<class 'float'>},{<class 'float'>},1,1,8,8,residential-area,Keep,None
362,ZABEOTYP,0.0000,0.0000,{<class 'int'>},{<class 'int'>},1,1,6,6,typification of energy consumers,Keep,None
363,ANREDE_KZ,0.0000,0.0000,{<class 'int'>},{<class 'int'>},1,1,2,2,gender,Keep,None


### Analyze nan values

In [18]:
nan_threshold = 0.3
meta_df.loc[(meta_df["na_percentage_azdias"] > nan_threshold) | (meta_df["na_percentage_customers"] > nan_threshold)]

,column_name,na_percentage_azdias,na_percentage_customers,dtypes_azdias,dtypes_customers,n_dtypes_azdias,n_dtypes_customers,unique_values_azdias,unique_values_customers,description,action,reason
3,ALTER_KIND1,0.9090,0.9386,{<class 'float'>},{<class 'float'>},1,1,17,17,No description available.,Keep,None
4,ALTER_KIND2,0.9669,0.9734,{<class 'float'>},{<class 'float'>},1,1,17,17,No description available.,Keep,None
5,ALTER_KIND3,0.9931,0.9933,{<class 'float'>},{<class 'float'>},1,1,15,14,No description available.,Keep,None
6,ALTER_KIND4,0.9986,0.9988,{<class 'float'>},{<class 'float'>},1,1,12,10,No description available.,Keep,None
99,EXTSEL992,0.7340,0.4450,{<class 'float'>},{<class 'float'>},1,1,56,56,No description available.,Keep,None
299,KK_KUNDENTYP,0.6560,0.5841,{<class 'float'>},{<class 'float'>},1,1,6,6,No description available.,Keep,None


In [19]:
# Every column with a nan count over 30 percent has also no description. These columns will be dropped from dataset:
print(f"Shape old df_azdias: {df_azdias.shape}")
print(f"Shape old df_customers: {df_customers.shape}")
df_azdias = df_azdias.drop(columns=["ALTER_KIND1", "ALTER_KIND2", "ALTER_KIND3", "ALTER_KIND4", "EXTSEL992", "KK_KUNDENTYP"])
df_customers = df_customers.drop(columns=["ALTER_KIND1", "ALTER_KIND2", "ALTER_KIND3", "ALTER_KIND4", "EXTSEL992", "KK_KUNDENTYP"])
print(f"Shape new df_azdias: {df_azdias.shape}")
print(f"Shape new df_customers: {df_customers.shape}")

Shape old df_azdias: (891221, 365)
Shape old df_customers: (191652, 368)
Shape new df_azdias: (891221, 359)
Shape new df_customers: (191652, 362)


In [20]:
drop_indices = meta_df.loc[(meta_df["na_percentage_azdias"] > nan_threshold) | (meta_df["na_percentage_customers"] > nan_threshold)].index

In [21]:
for i in drop_indices:
  meta_df.at[i, "action"] = "Drop"
  meta_df.at[i, "reason"] = "Too many nans"
  
meta_df.loc[(meta_df["na_percentage_azdias"] > nan_threshold) | (meta_df["na_percentage_customers"] > nan_threshold)]

,column_name,na_percentage_azdias,na_percentage_customers,dtypes_azdias,dtypes_customers,n_dtypes_azdias,n_dtypes_customers,unique_values_azdias,unique_values_customers,description,action,reason
3,ALTER_KIND1,0.9090,0.9386,{<class 'float'>},{<class 'float'>},1,1,17,17,No description available.,Drop,Too many nans
4,ALTER_KIND2,0.9669,0.9734,{<class 'float'>},{<class 'float'>},1,1,17,17,No description available.,Drop,Too many nans
5,ALTER_KIND3,0.9931,0.9933,{<class 'float'>},{<class 'float'>},1,1,15,14,No description available.,Drop,Too many nans
6,ALTER_KIND4,0.9986,0.9988,{<class 'float'>},{<class 'float'>},1,1,12,10,No description available.,Drop,Too many nans
99,EXTSEL992,0.7340,0.4450,{<class 'float'>},{<class 'float'>},1,1,56,56,No description available.,Drop,Too many nans
299,KK_KUNDENTYP,0.6560,0.5841,{<class 'float'>},{<class 'float'>},1,1,6,6,No description available.,Drop,Too many nans


In [22]:
# Analyze nas by rows
df_azdias["row_nan_percentage"] = pd.Series(df_azdias.index).apply(lambda x: df_azdias.iloc[x, :].isna().sum()/df_azdias.shape[1])
df_azdias["row_nan_percentage"].apply(lambda x: round(x, 1)).value_counts()

0.0    746728
0.7     73599
0.2     33965
0.6     20112
0.5      5523
0.1      4603
0.3      3645
0.4      3046
Name: row_nan_percentage, dtype: int64

In [23]:
row_threshold = 0.3
print(f"Before: {df_azdias.shape}")
df_azdias = df_azdias.loc[df_azdias["row_nan_percentage"] < row_threshold]
df_azdias = df_azdias.drop(columns=["row_nan_percentage"])
print(f"After: {df_azdias.shape}")

Before: (891221, 360)
After: (785421, 359)


### Identify columns with mixed data types

In [24]:
meta_df.loc[(meta_df["n_dtypes_azdias"] > 1) | (meta_df["n_dtypes_customers"] > 1)]

,column_name,na_percentage_azdias,na_percentage_customers,dtypes_azdias,dtypes_customers,n_dtypes_azdias,n_dtypes_customers,unique_values_azdias,unique_values_customers,description,action,reason
16,CAMEO_DEU_2015,0.1111,0.2631,"{<class 'float'>, <class 'str'>}","{<class 'float'>, <class 'str'>}",2,2,45,45,CAMEO classification 2015 - detailled classifi...,Keep,None
17,CAMEO_DEUG_2015,0.1111,0.2631,"{<class 'float'>, <class 'str'>}","{<class 'float'>, <class 'str'>}",2,2,10,10,CAMEO classification 2015 - Uppergroup,Keep,None
18,CAMEO_INTL_2015,0.1111,0.2631,"{<class 'float'>, <class 'str'>}","{<class 'float'>, <class 'str'>}",2,2,22,22,No description available.,Keep,None
60,D19_LETZTER_KAUF_BRANCHE,0.2885,0.2489,"{<class 'float'>, <class 'str'>}","{<class 'float'>, <class 'str'>}",2,2,35,35,No description available.,Keep,None
96,EINGEFUEGT_AM,0.1045,0.2605,"{<class 'float'>, <class 'str'>}","{<class 'float'>, <class 'str'>}",2,2,5162,3034,No description available.,Keep,None
316,OST_WEST_KZ,0.1045,0.2605,"{<class 'float'>, <class 'str'>}","{<class 'float'>, <class 'str'>}",2,2,2,2,flag indicating the former GDR/FRG,Keep,None


In [25]:
# CAMEO_DEU_2015
print(set(df_azdias["CAMEO_DEU_2015"]))
df_attributes_values.loc[df_attributes_values["attribute"] == "CAMEO_DEU_2015"]

{nan, '7C', '6D', '7D', '1B', '1E', '8B', '3C', '2C', '5B', '1A', '5D', '6C', '9C', '9A', '5A', '8D', '6F', '2A', 'XX', '5F', '3D', '3B', '9B', '7B', '1D', '7A', '8C', '6B', '5C', '9D', '8A', '2B', '4C', '1C', '4E', '5E', '6E', '9E', '4B', '3A', '7E', '6A', '4A', '2D', '4D'}


,attribute,description,value,meaning
62,CAMEO_DEU_2015,CAMEO classification 2015 - detailled classifi...,1A,Work-Life-Balance
63,CAMEO_DEU_2015,CAMEO classification 2015 - detailled classifi...,1B,Wealthy Best Ager
64,CAMEO_DEU_2015,CAMEO classification 2015 - detailled classifi...,1C,Successful Songwriter
65,CAMEO_DEU_2015,CAMEO classification 2015 - detailled classifi...,1D,Old Nobility
66,CAMEO_DEU_2015,CAMEO classification 2015 - detailled classifi...,1E,City Nobility
67,CAMEO_DEU_2015,CAMEO classification 2015 - detailled classifi...,2A,Cottage Chic
68,CAMEO_DEU_2015,CAMEO classification 2015 - detailled classifi...,2B,Noble Jogger
69,CAMEO_DEU_2015,CAMEO classification 2015 - detailled classifi...,2C,Established gourmet
70,CAMEO_DEU_2015,CAMEO classification 2015 - detailled classifi...,2D,Fine Management
71,CAMEO_DEU_2015,CAMEO classification 2015 - detailled classifi...,3A,Career & Family


In [26]:
# This column has too many possible categories which is too fine-granular. These columns will be droped.
meta_df.at[16, "action"] = "Drop"
meta_df.at[16, "reason"] = "Too many categories"

print(f"Shape old df_azdias: {df_azdias.shape}")
print(f"Shape old df_customers: {df_customers.shape}")
df_azdias = df_azdias.drop(columns=["CAMEO_DEU_2015"])
df_customers = df_customers.drop(columns=["CAMEO_DEU_2015"])
print(f"Shape new df_azdias: {df_azdias.shape}")
print(f"Shape new df_customers: {df_customers.shape}")

Shape old df_azdias: (785421, 359)
Shape old df_customers: (191652, 362)
Shape new df_azdias: (785421, 358)
Shape new df_customers: (191652, 361)


In [27]:
# CAMEO_DEU_2015
print(set(df_azdias["CAMEO_DEUG_2015"]))
df_attributes_values.loc[df_attributes_values["attribute"] == "CAMEO_DEUG_2015"]

{nan, '3', '7', '9', '6', 'X', '4', '1', '2', '8', '5'}


,attribute,description,value,meaning
52,CAMEO_DEUG_2015,CAMEO classification 2015 - Uppergroup,-1,unknown
53,CAMEO_DEUG_2015,CAMEO classification 2015 - Uppergroup,1,upper class
54,CAMEO_DEUG_2015,CAMEO classification 2015 - Uppergroup,2,upper middleclass
55,CAMEO_DEUG_2015,CAMEO classification 2015 - Uppergroup,3,established middleclasse
56,CAMEO_DEUG_2015,CAMEO classification 2015 - Uppergroup,4,consumption-oriented middleclass
57,CAMEO_DEUG_2015,CAMEO classification 2015 - Uppergroup,5,active middleclass
58,CAMEO_DEUG_2015,CAMEO classification 2015 - Uppergroup,6,low-consumption middleclass
59,CAMEO_DEUG_2015,CAMEO classification 2015 - Uppergroup,7,lower middleclass
60,CAMEO_DEUG_2015,CAMEO classification 2015 - Uppergroup,8,working class
61,CAMEO_DEUG_2015,CAMEO classification 2015 - Uppergroup,9,urban working class


In [28]:
# These column more coarsely granular
# Thus, it will be kept
# But there are some values ('X') that are not in the description
X_len = len(df_azdias.loc[df_azdias["CAMEO_DEUG_2015"] == 'X']["CAMEO_DEUG_2015"])
print(f"Number of rows with value 'X': {X_len}")

Number of rows with value 'X': 347


In [29]:
# Change 'X' values to np.nan first (later handle the nans)
df_azdias["CAMEO_DEUG_2015"] = df_azdias["CAMEO_DEUG_2015"].apply(lambda x: np.nan if x == 'X' else x)
df_customers["CAMEO_DEUG_2015"] = df_customers["CAMEO_DEUG_2015"].apply(lambda x: np.nan if x == 'X' else x)

# Check if it worked
print(set(df_azdias["CAMEO_DEUG_2015"]))

meta_df.at[17, "action"] = "Engineered"
meta_df.at[17, "reason"] = "Deleted values that werent in the description"

print(df_azdias["CAMEO_DEUG_2015"].value_counts())

{nan, '3', '7', '9', '6', '4', '1', '2', '8', '5'}
8    132942
9    106296
6    104762
4    102620
3     85436
2     82327
7     76817
5     54353
1     35870
Name: CAMEO_DEUG_2015, dtype: int64


In [30]:
# CAMEO_INTL_2015
print(set(df_azdias["CAMEO_INTL_2015"]))
df_attributes_values.loc[df_attributes_values["attribute"] == "CAMEO_INTL_2015"]

{nan, '43', '15', '54', '22', '32', 'XX', '13', '31', '12', '23', '25', '35', '52', '45', '34', '24', '44', '41', '33', '51', '55', '14'}


,attribute,description,value,meaning


In [31]:
# It seems that CAMEO_INTL_2015 is not in the df_attributes_values
# Check if there can be something siminliar found
df_attributes_values.loc[df_attributes_values["value"] == 44]

,attribute,description,value,meaning
125,CAMEO_DEUINTL_2015,(each German CAMEO code belongs to one interna...,44,Less Affluent Households-Older Families & Matu...


In [32]:
df_attributes_values.loc[df_attributes_values["attribute"] == "CAMEO_DEUINTL_2015"]

,attribute,description,value,meaning
106,CAMEO_DEUINTL_2015,CAMEO classification 2015 - international typo...,-1,unknown
107,CAMEO_DEUINTL_2015,(each German CAMEO code belongs to one interna...,11,Wealthy Households-Pre-Family Couples & Singles
108,CAMEO_DEUINTL_2015,(each German CAMEO code belongs to one interna...,12,Wealthy Households-Young Couples With Children
109,CAMEO_DEUINTL_2015,(each German CAMEO code belongs to one interna...,13,Wealthy Households-Families With School Age Ch...
110,CAMEO_DEUINTL_2015,(each German CAMEO code belongs to one interna...,14,Wealthy Households-Older Families & Mature Co...
111,CAMEO_DEUINTL_2015,(each German CAMEO code belongs to one interna...,15,Wealthy Households-Elders In Retirement
112,CAMEO_DEUINTL_2015,(each German CAMEO code belongs to one interna...,21,Prosperous Households-Pre-Family Couples & Sin...
113,CAMEO_DEUINTL_2015,(each German CAMEO code belongs to one interna...,22,Prosperous Households-Young Couples With Children
114,CAMEO_DEUINTL_2015,(each German CAMEO code belongs to one interna...,23,Prosperous Households-Families With School Age...
115,CAMEO_DEUINTL_2015,(each German CAMEO code belongs to one interna...,24,Prosperous Households-Older Families & Mature ...


In [33]:
# CAMEO_DEUINTL_2015 is not in the original demographic dataframe
# The original CAMEO_INTL_2015 column can be engineered to be less fine-granular
# 1 Wealthy
# 2 Prosperous
# 3 Comfortable
# 4 Less Affluente
# 5 Poorer
# Also the 'X' values will be changed to be np.nan
df_azdias["CAMEO_INTL_2015"] = df_azdias["CAMEO_INTL_2015"].apply(engineer_CAMEO_INTL_2015)
df_customers["CAMEO_INTL_2015"] = df_customers["CAMEO_INTL_2015"].apply(engineer_CAMEO_INTL_2015)
# Check if it worked
print(set(df_azdias["CAMEO_INTL_2015"]))

meta_df.at[18, "action"] = "Engineered"
meta_df.at[18, "reason"] = "Deleted values that werent in the description. Reduced categories"

print(df_azdias["CAMEO_INTL_2015"].value_counts())

{nan, '3', '1', '4', '2', '5'}
5    220488
2    188056
4    187602
1    118197
3     67080
Name: CAMEO_INTL_2015, dtype: int64


In [34]:
# D19_LETZTER_KAUF_BRANCHE
print(set(df_azdias["D19_LETZTER_KAUF_BRANCHE"]))
df_attributes_values.loc[df_attributes_values["attribute"] == "D19_LETZTER_KAUF_BRANCHE"]

{nan, 'D19_DROGERIEARTIKEL', 'D19_LOTTO', 'D19_TELKO_MOBILE', 'D19_NAHRUNGSERGAENZUNG', 'D19_BILDUNG', 'D19_BANKEN_GROSS', 'D19_GARTEN', 'D19_FREIZEIT', 'D19_HANDWERK', 'D19_VOLLSORTIMENT', 'D19_DIGIT_SERV', 'D19_BANKEN_LOKAL', 'D19_ENERGIE', 'D19_KINDERARTIKEL', 'D19_BANKEN_REST', 'D19_TECHNIK', 'D19_VERSAND_REST', 'D19_HAUS_DEKO', 'D19_BIO_OEKO', 'D19_BANKEN_DIREKT', 'D19_BUCH_CD', 'D19_WEIN_FEINKOST', 'D19_KOSMETIK', 'D19_SAMMELARTIKEL', 'D19_RATGEBER', 'D19_SCHUHE', 'D19_LEBENSMITTEL', 'D19_REISEN', 'D19_SONSTIGE', 'D19_UNBEKANNT', 'D19_BEKLEIDUNG_GEH', 'D19_TELKO_REST', 'D19_VERSICHERUNGEN', 'D19_BEKLEIDUNG_REST', 'D19_TIERARTIKEL'}


,attribute,description,value,meaning


In [35]:
# This column has too many possible categories which is too fine-granular. These columns will be droped.
meta_df.at[60, "action"] = "Drop"
meta_df.at[60, "reason"] = "Too many categories"

print(f"Shape old df_azdias: {df_azdias.shape}")
print(f"Shape old df_customers: {df_customers.shape}")
df_azdias = df_azdias.drop(columns=["D19_LETZTER_KAUF_BRANCHE"])
df_customers = df_customers.drop(columns=["D19_LETZTER_KAUF_BRANCHE"])
print(f"Shape new df_azdias: {df_azdias.shape}")
print(f"Shape new df_customers: {df_customers.shape}")

Shape old df_azdias: (785421, 358)
Shape old df_customers: (191652, 361)
Shape new df_azdias: (785421, 357)
Shape new df_customers: (191652, 360)


In [36]:
# EINGEFUEGT_AM
print(set(df_azdias["EINGEFUEGT_AM"]))
df_attributes_values.loc[df_attributes_values["attribute"] == "EINGEFUEGT_AM"]

{'2003-06-13 00:00:00', '1994-09-14 00:00:00', '1996-12-13 00:00:00', '2004-04-16 00:00:00', '2009-01-14 00:00:00', '2007-05-08 00:00:00', '1997-08-21 00:00:00', '2003-07-14 00:00:00', '2002-06-07 00:00:00', '2009-09-16 00:00:00', '2002-08-26 00:00:00', '2006-10-09 00:00:00', '2004-07-30 00:00:00', '2006-09-30 00:00:00', '2011-08-09 00:00:00', '2004-02-13 00:00:00', '2010-02-17 00:00:00', '2006-03-15 00:00:00', '2009-10-26 00:00:00', '1994-02-10 00:00:00', '2000-05-29 00:00:00', '2004-07-23 00:00:00', '1994-02-18 00:00:00', '2002-12-18 00:00:00', '2008-12-26 00:00:00', '2007-02-22 00:00:00', '2012-03-13 00:00:00', '2011-11-10 00:00:00', '2012-10-21 00:00:00', '2005-08-26 00:00:00', '2010-10-18 00:00:00', '2003-10-21 00:00:00', '2007-10-31 00:00:00', '2002-06-26 00:00:00', '2002-07-24 00:00:00', '2012-04-11 00:00:00', '2010-03-22 00:00:00', '2008-06-30 00:00:00', '2003-04-11 00:00:00', '1994-10-10 00:00:00', '2005-02-17 00:00:00', '1994-12-22 00:00:00', '1994-05-18 00:00:00', '1998-01-1

,attribute,description,value,meaning


In [37]:
# This is a date value and we will just keep the year since it holds the most information
df_azdias["EINGEFUEGT_AM"] = df_azdias["EINGEFUEGT_AM"].apply(lambda x: str(x)[:4])
df_customers["EINGEFUEGT_AM"] = df_customers["EINGEFUEGT_AM"].apply(lambda x: str(x)[:4])
# Check if it worked
print(set(df_azdias["EINGEFUEGT_AM"]))
meta_df.at[96, "action"] = "Engineered"
meta_df.at[96, "reason"] = "Just keep the year"

print(df_azdias["EINGEFUEGT_AM"].value_counts())

{'1997', '2001', '1991', '1992', '2013', '1995', '2003', '1993', '2008', '2009', '1999', '1994', '2007', '2004', '2005', '2011', '2000', '1996', '2012', '2006', '1998', '2002', '2010'}
1992    573892
1995     42226
1996     29539
1994     28647
1993     23981
1997     19542
2005     14005
2003     10074
2004      6190
2000      5815
2006      5276
2007      3631
1998      3428
2008      3179
2002      3108
2009      3008
2012      2771
2011      2553
2010      1659
2001      1626
1999      1050
2013       218
1991         3
Name: EINGEFUEGT_AM, dtype: int64


In [38]:
# OST_WEST_KZ
print(set(df_azdias["OST_WEST_KZ"]))
df_attributes_values.loc[df_attributes_values["attribute"] == "OST_WEST_KZ"]

{'O', 'W'}


,attribute,description,value,meaning
2014,OST_WEST_KZ,flag indicating the former GDR/FRG,-1,unknown
2015,OST_WEST_KZ,flag indicating the former GDR/FRG,O,East (GDR)
2016,OST_WEST_KZ,flag indicating the former GDR/FRG,W,West (FRG)


In [39]:
# Since there are only 2 classes they can be changed to binary
df_azdias["OST_WEST_KZ"] = df_azdias["OST_WEST_KZ"].apply(engineer_OST_WEST_KZ)
df_customers["OST_WEST_KZ"] = df_customers["OST_WEST_KZ"].apply(engineer_OST_WEST_KZ)

# Check if it worked
print(set(df_azdias["OST_WEST_KZ"])) 

meta_df.at[316, "action"] = "Engineered"
meta_df.at[316, "reason"] = "Changed to binary"

print(df_azdias["OST_WEST_KZ"].value_counts())

{0, 1}
1    619287
0    166134
Name: OST_WEST_KZ, dtype: int64


### Save interim results

In [41]:
df_azdias.shape

(785421, 357)

In [56]:
df_azdias.iloc[0:80000,:].to_csv("df_azdias_cleaned_1.csv", index=False)
df_azdias_cleaned_file = drive.CreateFile({'title' : "df_azdias_cleaned_1.csv"})
df_azdias_cleaned_file.SetContentFile("df_azdias_cleaned_1.csv")
df_azdias_cleaned_file.Upload()
drive.CreateFile({'id': df_azdias_cleaned_file.get("df_azdias_cleaned_1.csv")})

GoogleDriveFile({'id': None})

In [57]:
df_azdias.iloc[80000:160000,:].to_csv("df_azdias_cleaned_2.csv", index=False)
df_azdias_cleaned_file = drive.CreateFile({'title' : "df_azdias_cleaned_2.csv"})
df_azdias_cleaned_file.SetContentFile("df_azdias_cleaned_2.csv")
df_azdias_cleaned_file.Upload()
drive.CreateFile({'id': df_azdias_cleaned_file.get("df_azdias_cleaned_2.csv")})

GoogleDriveFile({'id': None})

In [58]:
df_azdias.iloc[160000:240000,:].to_csv("df_azdias_cleaned_3.csv", index=False)
df_azdias_cleaned_file = drive.CreateFile({'title' : "df_azdias_cleaned_3.csv"})
df_azdias_cleaned_file.SetContentFile("df_azdias_cleaned_3.csv")
df_azdias_cleaned_file.Upload()
drive.CreateFile({'id': df_azdias_cleaned_file.get("df_azdias_cleaned_3.csv")})

GoogleDriveFile({'id': None})

In [59]:
df_azdias.iloc[240000:320000,:].to_csv("df_azdias_cleaned_4.csv", index=False)
df_azdias_cleaned_file = drive.CreateFile({'title' : "df_azdias_cleaned_4.csv"})
df_azdias_cleaned_file.SetContentFile("df_azdias_cleaned_4.csv")
df_azdias_cleaned_file.Upload()
drive.CreateFile({'id': df_azdias_cleaned_file.get("df_azdias_cleaned_4.csv")})

GoogleDriveFile({'id': None})

In [60]:
df_azdias.iloc[320000:400000,:].to_csv("df_azdias_cleaned_5.csv", index=False)
df_azdias_cleaned_file = drive.CreateFile({'title' : "df_azdias_cleaned_5.csv"})
df_azdias_cleaned_file.SetContentFile("df_azdias_cleaned_5.csv")
df_azdias_cleaned_file.Upload()
drive.CreateFile({'id': df_azdias_cleaned_file.get("df_azdias_cleaned_5.csv")})

GoogleDriveFile({'id': None})

In [61]:
df_azdias.iloc[400000:480000,:].to_csv("df_azdias_cleaned_6.csv", index=False)
df_azdias_cleaned_file = drive.CreateFile({'title' : "df_azdias_cleaned_6.csv"})
df_azdias_cleaned_file.SetContentFile("df_azdias_cleaned_6.csv")
df_azdias_cleaned_file.Upload()
drive.CreateFile({'id': df_azdias_cleaned_file.get("df_azdias_cleaned_6.csv")})

GoogleDriveFile({'id': None})

In [62]:
df_azdias.iloc[480000:560000,:].to_csv("df_azdias_cleaned_7.csv", index=False)
df_azdias_cleaned_file = drive.CreateFile({'title' : "df_azdias_cleaned_7.csv"})
df_azdias_cleaned_file.SetContentFile("df_azdias_cleaned_7.csv")
df_azdias_cleaned_file.Upload()
drive.CreateFile({'id': df_azdias_cleaned_file.get("df_azdias_cleaned_7.csv")})

GoogleDriveFile({'id': None})

In [63]:
df_azdias.iloc[560000:640000,:].to_csv("df_azdias_cleaned_8.csv", index=False)
df_azdias_cleaned_file = drive.CreateFile({'title' : "df_azdias_cleaned_8.csv"})
df_azdias_cleaned_file.SetContentFile("df_azdias_cleaned_8.csv")
df_azdias_cleaned_file.Upload()
drive.CreateFile({'id': df_azdias_cleaned_file.get("df_azdias_cleaned_8.csv")})

GoogleDriveFile({'id': None})

In [64]:
df_azdias.iloc[640000:720000,:].to_csv("df_azdias_cleaned_9.csv", index=False)
df_azdias_cleaned_file = drive.CreateFile({'title' : "df_azdias_cleaned_9.csv"})
df_azdias_cleaned_file.SetContentFile("df_azdias_cleaned_9.csv")
df_azdias_cleaned_file.Upload()
drive.CreateFile({'id': df_azdias_cleaned_file.get("df_azdias_cleaned_9.csv")})

GoogleDriveFile({'id': None})

In [65]:
df_azdias.iloc[720000:,:].to_csv("df_azdias_cleaned_10.csv", index=False)
df_azdias_cleaned_file = drive.CreateFile({'title' : "df_azdias_cleaned_10.csv"})
df_azdias_cleaned_file.SetContentFile("df_azdias_cleaned_10.csv")
df_azdias_cleaned_file.Upload()
drive.CreateFile({'id': df_azdias_cleaned_file.get("df_azdias_cleaned_10.csv")})

GoogleDriveFile({'id': None})

In [42]:
df_customers.shape

(191652, 360)

In [66]:
df_customers.iloc[0:80000,:].to_csv("df_customers_cleaned_1.csv", index=False)
df_customers_cleaned_file_1 = drive.CreateFile({'title' : "df_customers_cleaned_1.csv"})
df_customers_cleaned_file_1.SetContentFile("df_customers_cleaned_1.csv")
df_customers_cleaned_file_1.Upload()
drive.CreateFile({'id': df_customers_cleaned_file_1.get("df_customers_cleaned_1.csv")})

GoogleDriveFile({'id': None})

In [67]:
df_customers.iloc[80000:160000,:].to_csv("df_customers_cleaned_2.csv", index=False)
df_customers_cleaned_file_1 = drive.CreateFile({'title' : "df_customers_cleaned_2.csv"})
df_customers_cleaned_file_1.SetContentFile("df_customers_cleaned_2.csv")
df_customers_cleaned_file_1.Upload()
drive.CreateFile({'id': df_customers_cleaned_file_1.get("df_customers_cleaned_2.csv")})

GoogleDriveFile({'id': None})

In [68]:
df_customers.iloc[160000:,:].to_csv("df_customers_cleaned_3.csv", index=False)
df_customers_cleaned_file_1 = drive.CreateFile({'title' : "df_customers_cleaned_3.csv"})
df_customers_cleaned_file_1.SetContentFile("df_customers_cleaned_3.csv")
df_customers_cleaned_file_1.Upload()
drive.CreateFile({'id': df_customers_cleaned_file_1.get("df_customers_cleaned_3.csv")})

GoogleDriveFile({'id': None})

In [43]:
meta_df.shape

(365, 12)

In [44]:
meta_df.to_csv("meta_df.csv", index=False)
meta_df_files = drive.CreateFile({'title' : "meta_df.csv"})
meta_df_files.SetContentFile("meta_df.csv")
meta_df_files.Upload()
drive.CreateFile({'id': meta_df_files.get("meta_df.csv")})

GoogleDriveFile({'id': None})

### Load interim results

In [73]:
# Get list of every available file in google drive
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
ticker=0
data=dict()
for file1 in file_list:
  if(ticker == 15):
    break
  else:
    data[file1['title']] = file1['id']
  ticker+=1
data

{'df_azdias_cleaned_1.csv': '1POZj6xeoJPx1_t-9-YEHpyZHqGGAaWjL',
 'df_azdias_cleaned_10.csv': '1R_ryleBPH_R1uoHvrPEhDF0LhVtQiMkL',
 'df_azdias_cleaned_2.csv': '1IWibD14sItQ9z9i9VPQtD33jYzOnlwTI',
 'df_azdias_cleaned_3.csv': '1uPOdYseNcqYTCb_-XNXAHFnj2U76r8as',
 'df_azdias_cleaned_4.csv': '1jwkzS6T9hlSKIagb7GM-mslMQJl1badw',
 'df_azdias_cleaned_5.csv': '1jAAW1N3-PBtlKk3dGqqMMenPF30JpZL6',
 'df_azdias_cleaned_6.csv': '1AdGWWewqYyOrPKJ_75-acJuxSF-lEmCg',
 'df_azdias_cleaned_7.csv': '1J2f8ldpqH_5zVXzZ9hBYDLjOVdZD0FAN',
 'df_azdias_cleaned_8.csv': '1V9iyfiXF-rwRAIwznVcYDkVacFy7Xaxa',
 'df_azdias_cleaned_9.csv': '17if_VSLxvnPqw-cQRCWwppP_4pbUpa2p',
 'df_customers_cleaned_1.csv': '1Eb8SkudrXqF8smDigFHDMUIJmsPACqYm',
 'df_customers_cleaned_2.csv': '1vDuhzRuLUQXCnzl7njDcVZ6DmDbCZ5_A',
 'df_customers_cleaned_3.csv': '1yP3Pymw4Fkp9K6_n-uQ_Su6YFgGmirBo',
 'meta_df.csv': '1Phry8vukY_1Dp-weiHY5288jhwXEb1xG'}

In [75]:
for k, v in data.items():
  print(k)

df_customers_cleaned_3.csv
df_customers_cleaned_2.csv
df_customers_cleaned_1.csv
df_azdias_cleaned_10.csv
df_azdias_cleaned_9.csv
df_azdias_cleaned_8.csv
df_azdias_cleaned_7.csv
df_azdias_cleaned_6.csv
df_azdias_cleaned_5.csv
df_azdias_cleaned_4.csv
df_azdias_cleaned_3.csv
df_azdias_cleaned_2.csv
df_azdias_cleaned_1.csv
meta_df.csv


In [ ]:
download_data = drive.CreateFile({'id': '135UcsbUyLcwS16QHa7lFcXHKhuy7HwWF'})
download_data.GetContentFile('azdias.csv')  
df_azdias = pd.read_csv('azdias.csv', engine="python", sep=';')

print(df_azdias.shape)